# RemoteSqlite

RemoteSqlite is a way to easily access a remote Sqlite database and perform queries, including SELECT and INSERT operations

The remote database is referenced through a PyFilesystem2 url, such as:

- `osfs://path/to/file`
- `s3://path/to/file`

You can perform `pull` and `push` operations that transfer the database to a local temp directory

See below for examples of how to use

In [2]:
class RemoteSqlite:
    def __init__(self, fspath):
        import sqlite3
        self.fspath = fspath
        self.localpath = self.pull(fspath)
        self.con = sqlite3.connect(self.localpath, detect_types=sqlite3.PARSE_DECLTYPES)
        self.con.row_factory = sqlite3.Row
    def __del__(self):
        self.con.close()
    def pull(self, fspath):
        import fs
        remotefs = fs.open_fs(fs.path.dirname(fspath))
        tempfs = fs.open_fs('osfs:///tmp')
        filename = fs.path.basename(fspath)
        fs.copy.copy_file_if_newer(remotefs, filename, tempfs, filename)
        return tempfs.getsyspath(filename)
    def push(self, fspath):
        import fs
        remotefs = fs.open_fs(fs.path.dirname(fspath))
        tempfs = fs.open_fs('osfs:///tmp')
        filename = fs.path.basename(fspath)
        fs.copy.copy_file_if_newer(tempfs, filename, remotefs, filename)
        return fspath
    def get_count(self, tbl_name):
        return db.select(f"""SELECT COUNT(*) FROM `{tbl_name}`""")[0]['COUNT(*)']
    def get_counts(self):
        tables = self.select("""SELECT tbl_name FROM sqlite_master WHERE type='table'""")
        return [{t['tbl_name']: self.get_count(t['tbl_name'])} for t in tables]
    def select(self, select_statement='SELECT * FROM sqlite_master'):
        cur = self.con.cursor()
        cur.execute(select_statement)
        records = [dict(row) for row in cur.fetchall()]
        return records
    def insert(self, tbl_name, records):
        cur = self.con.cursor()
        for record in records:
            field_names = ','.join([f'"{k}"' for k in record.keys()])
            placeholders = ','.join(['?' for k in record.keys()])
            insert_statement = f'INSERT INTO "{tbl_name}" ({field_names}) VALUES ({placeholders})'
            values = tuple(record.values())
            cur.execute(insert_statement, values)
        self.con.commit()

In [4]:
db = RemoteSqlite('osfs:///Users/vbalasubramaniam/Downloads/Northwind_large.sqlite')

In [231]:
db.get_counts()

[{'Employee': 9},
 {'Category': 8},
 {'Customer': 91},
 {'Shipper': 3},
 {'Supplier': 29},
 {'Order': 16818},
 {'Product': 77},
 {'OrderDetail': 621883},
 {'CustomerCustomerDemo': 0},
 {'CustomerDemographic': 0},
 {'Region': 4},
 {'Territory': 53},
 {'EmployeeTerritory': 49},
 {'posts': 0},
 {'posts_data': 2},
 {'posts_idx': 0},
 {'posts_content': 0},
 {'posts_docsize': 0},
 {'posts_config': 1},
 {'order_search': 16818},
 {'order_search_data': 226},
 {'order_search_idx': 474},
 {'order_search_content': 16818},
 {'order_search_docsize': 16818},
 {'order_search_config': 1},
 {'test': 0},
 {'sqlite_sequence': 0}]

In [226]:
db.select("""SELECT * FROM Shipper""")


[{'Id': 1, 'CompanyName': 'Speedy Express', 'Phone': '(503) 555-9831'},
 {'Id': 2, 'CompanyName': 'United Package', 'Phone': '(503) 555-3199'},
 {'Id': 3, 'CompanyName': 'Federal Shipping', 'Phone': '(503) 555-9931'}]

In [227]:
new_records = [{'Id': 4, 'CompanyName': 'Speedy Express', 'Phone': '(503) 555-9831'},
 {'Id': 5, 'CompanyName': 'United Package', 'Phone': '(503) 555-3199'},
 {'Id': 6, 'CompanyName': 'Federal Shipping', 'Phone': '(503) 555-9931'}]

In [197]:
new_records

[{'Id': 4, 'CompanyName': 'Speedy Express', 'Phone': '(503) 555-9831'},
 {'Id': 5, 'CompanyName': 'United Package', 'Phone': '(503) 555-3199'},
 {'Id': 6, 'CompanyName': 'Federal Shipping', 'Phone': '(503) 555-9931'}]

In [223]:
db.insert('Shipper', new_records)

INSERT INTO "Shipper" ("Id","CompanyName","Phone") VALUES (?,?,?)
(4, 'Speedy Express', '(503) 555-9831')
INSERT INTO "Shipper" ("Id","CompanyName","Phone") VALUES (?,?,?)
(5, 'United Package', '(503) 555-3199')
INSERT INTO "Shipper" ("Id","CompanyName","Phone") VALUES (?,?,?)
(6, 'Federal Shipping', '(503) 555-9931')


In [225]:
db.con.execute('DELETE FROM Shipper WHERE Id >3')

In [238]:
create_statement = """CREATE TABLE test (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT,
  t TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)"""

In [239]:
db.con.execute(create_statement)

In [240]:
db.insert('test', [{'name': 'John'},{'name':'Matt'}])

INSERT INTO "test" ("name") VALUES (?)
('John',)
INSERT INTO "test" ("name") VALUES (?)
('Matt',)


In [241]:
db.select('SELECT * FROM test')

[{'id': 1, 'name': 'John', 't': datetime.datetime(2020, 3, 5, 14, 29, 15)},
 {'id': 2, 'name': 'Matt', 't': datetime.datetime(2020, 3, 5, 14, 29, 15)}]

In [237]:
db.con.execute('DROP TABLE test')